## Introduction

In this lab assessment, you'll practice your knowledge of JOIN statements and subqueries, using various types of joins and various methods for specifying the links between them. One of the main benefits of using a relational database is the table relations that define them which allow you to access and connect data together via shared columns. By writing more advanced SQL queries that utilize joins and subqueries you can provide a deeper and more granular level of analysis and data retrieval.

This assessment will continue looking at the familiar Northwind database that contains customer relationship management (CRM) data as well as employee and product data. You will take a deeper dive into this database in order to accomplish more advanced SQL queries that require you to access data from multiple tables at once. 

Imagine that you are working in an analyst role for the sales rep team. They have collaborated with the customer relations and the product teams to take a comprehensive look at the employee to customer pipeline in an attempt to find areas of improvement and potential growth. You have been asked to provide some specific data and statistics regarding this project.

## Learning Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Choose and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Database

The database will be the customer relationship management (CRM) database, which has the following ERD.

![Database-Schema.png](ERD.png)

### Connect to the database

In the cell below we have provided the code to import both pandas and sqlite3 as well as define and create the connection to the database you will use. Also displayed is the schema and table names from the database. Use this information in conjunction with the ERD image above to assist in creating your SQL Queries.

Major Hint: Look for the shared columns across tables you need to 'join' together.

In [1]:
# CodeGrade step0
# Run this cell without changes

# SQL Library and Pandas Library
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.sqlite')

pd.read_sql("""SELECT * FROM sqlite_master""", conn)

,type,name,tbl_name,rootpage,sql
0,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
1,table,offices,offices,32,"CREATE TABLE ""offices"" (\n\t""officeCode""\tINTE..."
2,table,customers,customers,57,"CREATE TABLE ""customers"" (\n\t""customerNumber""..."
3,table,employees,employees,35,"CREATE TABLE ""employees"" (\n\t""employeeNumber""..."
4,table,orderdetails,orderdetails,2,"CREATE TABLE ""orderdetails"" (\n\t""orderNumber""..."
5,table,orders,orders,27,"CREATE TABLE ""orders"" (\n\t""orderNumber""\tINTE..."
6,table,payments,payments,28,"CREATE TABLE ""payments"" (\n\t""customerNumber""\..."
7,table,products,products,4,"CREATE TABLE ""products"" (\n\t""productCode""\tTE..."


## Part 1: Join and Filter

### Step 1

The company would like to let Boston employees go remote but need to know more information about who is working in that office. Return the first and last names and the job titles for all employees in Boston.

In [2]:
# CodeGrade step1
# Replace None with your code
df_boston = pd.read_sql("""
                SELECT firstName, lastName, jobTitle, city
                FROM employees
                JOIN offices
                    USING(officeCode)
                WHERE city = 'Boston'                
;""", conn)
df_boston 

,firstName,lastName,jobTitle,city
0,Julie,Firrelli,Sales Rep,Boston
1,Steve,Patterson,Sales Rep,Boston


### Step 2

Recent downsizing and employee attrition have caused some mixups in office tracking and the company is worried they are supporting a 'ghost' location. Are there any offices that have zero employees?

In [3]:
# CodeGrade step2
# Replace None with your code
df_zero_emp = pd.read_sql("""
                SELECT officeCode, city,
                    COUNT(employeeNumber) AS num_empl
                FROM offices
                LEFT JOIN employees
                    USING(officeCode)                
                GROUP BY officeCode, city                        
                ORDER BY num_empl, city, officeCode
;""", conn)

# The second office (27) in Boston has cero employees
df_zero_emp

,officeCode,city,num_empl
0,27,Boston,0
1,2,Boston,2
2,7,London,2
3,3,NYC,2
4,5,Tokyo,2
5,6,Sydney,4
6,4,Paris,5
7,1,San Francisco,6


## Part 2: Type of Join

### Step 3

As a part of this larger analysis project the HR department is taking the time to audit employee records to make sure nothing is out of place and have asked you to produce a report of all employees. Return the employees first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [4]:
# CodeGrade step3
# Replace None with your code

df_employee = pd.read_sql("""
                SELECT firstName, lastName, city, state
                FROM employees
                LEFT JOIN offices
                    USING(officeCode)                
                ORDER BY firstName, lastName
;""", conn)

df_employee

,firstName,lastName,city,state
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,
7,Gerard,Hernandez,Paris,
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


### Step 4
The customer management and sales rep team know that they have several 'customers' in the system that have not placed any orders. They want to reach out to these customers with updated product catalogs to try and get them to place initial orders. Return all of the customer's contact information (first name, last name, and phone number) as well as their sales rep's employee number for any customer that has not placed an order. Sort the results alphabetically based on the contact's last name

There are several approaches you could take here, including a left join and filtering on null values or using a subquery to filter out customers who do have orders. In total there are 24 customers who have not placed an order.

In [5]:
# CodeGrade step4
# Replace None with your code

df_contacts = pd.read_sql("""
                SELECT customerName, contactFirstName, contactLastName, phone, salesRepEmployeeNumber, orderNumber
                FROM customers
                LEFT JOIN orders
                    USING(customerNumber)                
                WHERE orderNumber IS NULL
                ORDER BY contactFirstName, contactLastName
;""", conn)

df_contacts

,customerName,contactFirstName,contactLastName,phone,salesRepEmployeeNumber,orderNumber
0,ANG Resellers,Alejandra,Camino,(91) 745 6555,,None
1,"Feuer Online Stores, Inc",Alexander,Feuer,0342-555176,,None
2,"Kremlin Collectables, Co.",Alexander,Semenov,+7 812 293 0521,,None
3,"SAR Distributors, Co",Armand,Kuger,+27 21 550 3555,,None
4,Schuyler Imports,Bradley,Schuyler,+31 20 491 9555,,None
5,Precious Collectables,Braun,Urs,0452-076555,1702,None
6,"Asian Shopping Network, Co",Brydey,Walker,+612 9411 1555,,None
7,"Anton Designs, Ltd.",Carmen,Anton,+34 913 728555,,None
8,BG&E Collectables,Ed,Harrison,+41 26 425 50 01,,None
9,Mit Vergnügen & Co.,Hanna,Moos,0621-08555,,None


## Part 3: Built-in Function

### Step 5

The accounting team is auditing their figures and wants to make sure all customer payments are in alignment, they have asked you to produce a report of all the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. They have asked that these results be sorted in descending order by the payment amount.

Hint: A member of their team mentioned that they are not sure the 'amount' column is being stored as the right datatype so keep this in mind when sorting.

In [6]:
# CodeGrade step5
# Replace None with your code
df_payment = pd.read_sql("""
                SELECT customerName, contactFirstName, contactLastName, phone, CAST(amount AS FLOAT) AS amount_float, paymentDate
                FROM payments
                LEFT JOIN customers 
                    USING(customerNumber)                
                ORDER BY amount_float DESC
;""", conn)

df_payment

,customerName,contactFirstName,contactLastName,phone,amount_float,paymentDate
0,Euro+ Shopping Channel,Diego,Freyre,(91) 555 94 44,120166.58,2005-03-18
1,Euro+ Shopping Channel,Diego,Freyre,(91) 555 94 44,116208.40,2004-12-31
2,Mini Gifts Distributors Ltd.,Susan,Nelson,4155551450,111654.40,2003-08-15
3,"Dragon Souveniers, Ltd.",Eric,Natividad,+65 221 7555,105743.00,2003-12-26
4,Mini Gifts Distributors Ltd.,Susan,Nelson,4155551450,101244.59,2005-03-05
...,...,...,...,...,...,...
268,Atelier graphique,Carine,Schmitt,40.32.2555,1676.14,2004-12-18
269,Royale Belge,Pascale,Cartrain,(071) 23 67 2555,1627.56,2003-04-19
270,Baane Mini Imports,Jonas,Bergulfsen,07-98 9555,1491.38,2003-10-28
271,Royale Belge,Pascale,Cartrain,(071) 23 67 2555,1128.20,2003-08-22


## Part 4: Joining and Grouping

### Step 6

The sales rep team has noticed several key team members that stand out as having trustworthy business relations with their customers, reflected by high credit limits indicating more potential for orders. The team wants you to identify these 4 individuals. Return the employee number, first name, last name, and number of customers for employees whose customers have an average credit limit over 90k. Sort by number of customers from high to low.

In [7]:
# CodeGrade step6
# Replace None with your code
df_credit = pd.read_sql("""
                SELECT employeeNumber, firstName, lastName, 
                    COUNT(customerNumber) AS num_customers, 
                    AVG(creditLimit) AS avg_credit_limit
                FROM employees
                JOIN customers
                    ON employeeNumber = salesRepEmployeeNumber
                GROUP BY employeeNumber, firstName, lastName
                HAVING avg_credit_limit > 90000
                ORDER BY num_customers DESC
                LIMIT 4
;""", conn)

# It looks LIMIT 4 was not needed
df_credit

,employeeNumber,firstName,lastName,num_customers,avg_credit_limit
0,1501,Larry,Bott,8,91187.500000
1,1370,Gerard,Hernandez,7,91785.714286
2,1165,Leslie,Jennings,6,100433.333333
3,1612,Peter,Marsh,5,92080.000000


### Step 7

The product team is looking to create new model kits and wants to know which current products are selling the most in order to get an idea of what is popular. Return the product name and count the number of orders for each product as a column named 'numorders'. Also return a new column, 'totalunits', that sums up the total quantity of product sold (use the quantityOrdered column). Sort the results by the totalunits column, highest to lowest, to showcase the top selling products.

In [8]:
# CodeGrade step7
# Replace None with your code
df_product_sold = pd.read_sql("""
                SELECT 
                productName,
                COUNT(orderNumber) AS numorders,
                SUM(quantityOrdered) AS totalunits
                FROM products
                JOIN orderdetails
                    USING(productCode)
                GROUP BY productName
                ORDER BY totalunits DESC
;""", conn)

df_product_sold

,productName,numorders,totalunits
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


## Part 5: Multiple Joins

### Step 8

As a follow-up to the above question, the product team also wants to know how many different customers ordered each product to get an idea of market reach. Return the product name, code, and the total number of customers who have ordered each product, aliased as 'numpurchasers'. Sort the results by the highest  number of purchasers.

Hint: You might need to join more than 2 tables. Use DISTINCT to return unique/different values.

In [9]:
# CodeGrade step8
# Replace None with your code
df_total_customers = pd.read_sql("""
                SELECT
                productName,
                productCode,                               
                COUNT(DISTINCT customerNumber) AS numpurchasers
                FROM products
                JOIN orderdetails
                    USING(productCode)
                JOIN orders
                    USING(orderNumber)    
                JOIN customers
                    USING(customerNumber)                
                GROUP BY productName
                ORDER BY numpurchasers DESC
;""", conn)

df_total_customers

,productName,productCode,numpurchasers
0,1992 Ferrari 360 Spider red,S18_3232,40
1,Boeing X-32A JSF,S72_1253,27
2,1972 Alfa Romeo GTA,S10_4757,27
3,1952 Alpine Renault 1300,S10_1949,27
4,1934 Ford V8 Coupe,S18_2957,27
...,...,...,...
104,1958 Chevy Corvette Limited Edition,S24_2840,19
105,2002 Chevy Corvette,S24_3432,18
106,1969 Chevrolet Camaro Z28,S24_3191,18
107,1952 Citroen-15CV,S24_2887,18


### Step 9

The custom relations team is worried they are not staffing locations properly to account for customer volume. They want to know how many customers there are per office. Return the count as a column named 'n_customers'. Also return the office code and city.

In [10]:
# CodeGrade step9
# Replace None with your code
df_customers = pd.read_sql("""
                SELECT
                o.city,
                o.officeCode,                               
                COUNT(DISTINCT c.customerNumber) AS n_customers
                FROM offices AS o
                LEFT JOIN employees
                    USING(officeCode)
                LEFT JOIN customers AS c
                    ON employeeNumber = salesRepEmployeeNumber
                GROUP BY o.officeCode, o.city
                ORDER BY n_customers DESC
;""", conn)

# Office Boston:27 is cero customers and cero employees
df_customers

,city,officeCode,n_customers
0,Paris,4,29
1,London,7,17
2,NYC,3,15
3,San Francisco,1,12
4,Boston,2,12
5,Sydney,6,10
6,Tokyo,5,5
7,Boston,27,0


## Part 6: Subquery

### Step 10

Having looked at the results from above, the product team is curious to dig into the underperforming products. They want to ask members of the team who have sold these products about what kind of messaging was successful in getting a customer to buy these specific products. Using a subquery or common table expression (CTE), select the employee number, first name, last name, city of the office, and the office code for employees who sold products that have been ordered by fewer than 20 customers.

Hint: Start with the subquery, find all the products that have been ordered by 19 or less customers, consider adapting one of your previous queries.

In [11]:
pd.read_sql("""
    SELECT DISTINCT 
        e.employeeNumber,
        e.firstName,
        e.lastName,
        o.city,
        o.officeCode,
        p.productName,
        low_perf.num_customers
    FROM employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders ord ON c.customerNumber = ord.customerNumber
    JOIN orderdetails od ON ord.orderNumber = od.orderNumber
    JOIN products p ON od.productCode = p.productCode
    JOIN offices o ON e.officeCode = o.officeCode
    JOIN (
        SELECT 
            od2.productCode,
            COUNT(DISTINCT c2.customerNumber) AS num_customers
        FROM orderdetails od2
        JOIN orders o2 ON od2.orderNumber = o2.orderNumber
        JOIN customers c2 ON o2.customerNumber = c2.customerNumber
        GROUP BY od2.productCode
        HAVING COUNT(DISTINCT c2.customerNumber) < 20
    ) AS low_perf ON od.productCode = low_perf.productCode
    ORDER BY e.employeeNumber;
""", conn)

,employeeNumber,firstName,lastName,city,officeCode,productName,num_customers
0,1165,Leslie,Jennings,San Francisco,1,1949 Jaguar XK 120,18
1,1165,Leslie,Jennings,San Francisco,1,1958 Chevy Corvette Limited Edition,19
2,1165,Leslie,Jennings,San Francisco,1,1952 Citroen-15CV,18
3,1165,Leslie,Jennings,San Francisco,1,1969 Chevrolet Camaro Z28,18
4,1165,Leslie,Jennings,San Francisco,1,2002 Chevy Corvette,18
5,1166,Leslie,Thompson,San Francisco,1,1949 Jaguar XK 120,18
6,1166,Leslie,Thompson,San Francisco,1,1952 Citroen-15CV,18
7,1166,Leslie,Thompson,San Francisco,1,1969 Chevrolet Camaro Z28,18
8,1166,Leslie,Thompson,San Francisco,1,2002 Chevy Corvette,18
9,1188,Julie,Firrelli,Boston,2,1958 Chevy Corvette Limited Edition,19


### Close the connection

In [12]:
# Run this cell without changes

conn.close()